In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json

In [3]:
#CodeRead

data_folder = '../Data_contest/dataset/'


genome_scores_df=pd.read_csv(data_folder+'genome_scores.csv') # Large (500MB)
movies_df=pd.read_csv(data_folder+'movies.csv')
train_df=pd.read_csv(data_folder+'train.csv') # Large 500MB
validation_df = pd.read_csv(data_folder+'validation.csv') 








In [5]:
#CodeRead

# create movie rating dataset from train

# Feature vector for the 10000 movies, each with a 1128 dimensional vector. 
# If a movie doesn't appear in genome_scores we make it simply the 0 vector.
X=np.zeros((10000,1128)) 
movies_with_featvecs=set(genome_scores_df['movieId'])
# The average rating, for each of the movies in the training set. 
# -1 if it is not in the train set.
rating_movies = -1*np.ones(10000) 

# Each movie, is labelled +1 or -1 based on whetherr it is a comedy or not
comedy = np.zeros(10000) 

for i in range(10000):
    if i not in movies_with_featvecs:
        continue
    temp = genome_scores_df[genome_scores_df['movieId']==i]
    feat_vec= np.array(temp['relevance'])
    X[i,:]=feat_vec

for i in range(10000):
    temp = train_df[train_df['movieId']==i]
    if len(temp)==0:
        continue
    ratings_curr_movies = temp['rating']
    rating_movies[i] = np.mean(ratings_curr_movies)

for i in range(10000):
    temp = movies_df[movies_df['movieId']==i]
    if len(temp)==0:
        continue
    temp = temp['genres'].values[0]
    temp = temp.split('|')
    comedy[i] = 'Comedy' in temp
    comedy[i] = 2*comedy[i] -1


In [6]:
np.array(train_df['rating']).mean()

3.3601078084681526

# Problem 2: PCA and regression

Take the regression dataset below, and perform linear regression after doing PCA on the feature vector. 

For each K in [4,32,256,1024] take the top k components and report the mean squared error on the test set below. 

For each K you can choose the regularisation hyperparameter $\lambda$ for linear regression using a 80-20 split of the training set. 

For each K above, report the best lambda and the mean squared error for this best lambda in the cell below the next.




In [100]:
# CodeWrite

def pca(X_all, k=None):
    X_df = pd.DataFrame(X_all)
    cov_matrix = X_df.cov()
    u,s,vt = np.linalg.svd(cov_matrix)

    if k==None:
        k = X_all.shape[1]
    s_new = s[:k]
    u_new = u[:,:k]

    X_n = u_new.T.dot(X_df.T)
    X_new = X_n.T
    return X_new

def best_hyperparam(X_train, Y_train):
    split = int(0.8*X_train.shape[0])
    X_train1 = X_train[:split]
    X_val = X_train[split:]
    Y_train1 = Y_train[:split]
    Y_val = Y_train[split:]
    best_mse_loss = 1000
    best_reg_param = -1
    reg_params = [0,1e-5,1e-3,1e-1,1,1e1,1e3]
    for reg_param in reg_params:
        w = np.dot(np.matmul(np.linalg.inv(np.matmul(X_train1.T, X_train1)
                            + 0.5*reg_param*Y_train1.shape[0]*np.eye(X_train1.shape[1])), X_train1.T), Y_train1)
        Y_pred_val = np.dot(w, X_val.T)
        mse_val = np.mean((Y_pred_val - Y_val)**2)
        if mse_val < best_mse_loss:
            best_mse_loss = mse_val
            best_reg_param = reg_param
    return best_reg_param

X_all = X[rating_movies>0]
Y_all = rating_movies[rating_movies>0]

ks = [1,3,7,15,31,63]
for k in ks:
    X_new = pca(X_all,k=k)
    Y_new = Y_all

    X_train = np.array(X_new[:7000])
    Y_train = np.array(Y_new[:7000])
    X_test = np.array(X_new[7000:])
    Y_test = np.array(Y_new[7000:])


    best_reg_param = best_hyperparam(X_train, Y_train)
    w = np.dot(np.matmul(np.linalg.inv(np.matmul(X_train.T, X_train)
                                + 0.5*best_reg_param*Y_train.shape[0]*np.eye(X_train.shape[1])), X_train.T), Y_train)
    Y_pred_train = np.dot(w, X_train.T)
    Y_pred_test = np.dot(w, X_test.T)

    mse_train = np.mean((Y_pred_train - Y_train)**2)
    mse_test = np.mean((Y_pred_test - Y_test)**2)
#     print('K = {}, best_lambda = {}, mse train = {} , mse test = {}'.format(k,best_reg_param,mse_train,mse_test))
    print(k,'|', best_reg_param, '|', mse_train, '|',mse_test)

TextWrite cell. Report best lambda and the mean squared error on the test set, for different K here.

### PCA with Linear Regression

|      K     | Best Lambda | MSE Train  |  MSE Test  | 
|------------|-------------|------------|------------|
4 | 0.001 | 0.6545 | 0.8148 |
32 | 0 | 0.2925 | 0.4038 |
256 | 1e-05 | 0.1552 | 0.2463 |
1024 | 0.001 | 0.1412 | 0.2311 |

